# 1. dataset loading

In [9]:
# wine dataset을 사용

from sklearn.datasets import load_wine

wine_data = load_wine()
# wine_data는 data(features : X)와 target(label : y)으로 구성
X = wine_data.data
y = wine_data.target

print(X.shape)
print(y.shape)

(178, 13)
(178,)


# 2. kmeans clustering with numpy

In [14]:
import numpy as np

def euclidean_distance (x1 , x2):
    return np.sqrt(np.sum(x1-x2)**2)
# kmeans ( )  parameter : dataset(X) , cluster 수(k) , max_iter(무한루프 방지)
# (1) return할 값 : clustering의 결과 : labeling의 결과 --> labels
def kmeans (X , k , max_iters=100):
    # data의 갯수 : m <-- X.shape[0] , labeling의 결과 : n <-- X.shape[1]
    m , n = X.shape

    # pick k random points as centroids --> X 중 random하게 k개를 선택
    centroids_id = np.random.choice(m , k , replace=False) # m 개 중 k 개 뽑기 (3짜리 벡터)
    centroids = X[centroids_id] # 3X13

    for i in range(max_iters):
        # cluster the input points(X) to the nearest centroids
        # 모든 X에 대해서 각 centroids까지의 거리를 계산
        # 이 거리들 중에서 가장 최소값을 가진 centroid를 X의 label로 결정
        distances = [] # 모든 X에서 모든 centroids까지의 거리를 저장 : m x k 배열
        for s in range(m):
            distance = [] # s번째 X에서 모든 centroid까지의 거리
            for j in range(k):
                distance.append( np.sqrt(euclidean_distance(X[s] , centroids[j]))) # s와 centroids[]의 거리
            distances.append(distance)
        labels = np.argmin (distances , axis=1) # 가장 가까운 distance를 주는 centroid의 index

        # compute new centroids from the clusters
        # 같은 label을 갖는 pint들로부터 centroid를 다시 계산
        new_centroids = np.array([X[labels == s].mean(axis=0) for s in range(k)])

        # stable ? centroids = new_centroids
        if (np.array_equal( centroids , new_centroids)):
            print('break in {}'.format(i))
            break
        centroids = new_centroids
    return labels

In [34]:
num_cluster = 3
labels = kmeans (X , num_cluster , max_iters=50)

unique_labels , label_count = np.unique(labels , return_counts=True)
print(unique_labels)
print(label_count)

break in 6
[0 1 2]
[48 62 68]


## 2.1 결과 비교

In [24]:
# groundtruth의 label 수 계산
cnt0 = cnt1 = cnt2 = 0
for i in range(len(X)):
    cnt0 += (y[i]==0)
    cnt1 += (y[i]==1)
    cnt2 += (y[i]==2)
print(cnt0 , cnt1 , cnt2)

# random 하게 잡는 초기값 때문에 k-means는 돌릴 때마다 결과값이 달라질 수 있다.

59 71 48


# 3. kmeans clustering with sklearn

In [35]:
from sklearn.cluster import KMeans

kmeansk = KMeans (n_clusters=num_cluster)
kmeansk.fit(X) # kmean 결과는 kmeansk.labels_ 에 저장

unique_labels , label_count = np.unique(kmeansk.labels_ , return_counts=True)
print(unique_labels)
print(label_count)

[0 1 2]
[47 69 62]


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
